<a href="https://colab.research.google.com/github/hanzopgp/ArchitectApp/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --> Importations

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --> Global variables

In [7]:
SEQ_LEN = 64
HOLD_PERCENT_DIFF = 0.03

# --> Datasets on google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/Datasets/BTCPrice/btc_data"

BTCUSDT_01-01-2018_1m.csv  BTCUSDT_01-07-2020_1m.csv  BTCUSDT_02-01-2020_1m.csv
BTCUSDT_01-01-2019_1m.csv  BTCUSDT_01-08-2018_1m.csv  BTCUSDT_02-01-2021_1m.csv
BTCUSDT_01-01-2020_1m.csv  BTCUSDT_01-08-2019_1m.csv  BTCUSDT_02-02-2018_1m.csv
BTCUSDT_01-01-2021_1m.csv  BTCUSDT_01-08-2020_1m.csv  BTCUSDT_02-02-2019_1m.csv
BTCUSDT_01-02-2018_1m.csv  BTCUSDT_01-09-2017_1m.csv  BTCUSDT_02-02-2020_1m.csv
BTCUSDT_01-02-2019_1m.csv  BTCUSDT_01-09-2018_1m.csv  BTCUSDT_02-02-2021_1m.csv
BTCUSDT_01-02-2020_1m.csv  BTCUSDT_01-09-2019_1m.csv  BTCUSDT_02-03-2018_1m.csv
BTCUSDT_01-02-2021_1m.csv  BTCUSDT_01-09-2020_1m.csv  BTCUSDT_02-03-2019_1m.csv
BTCUSDT_01-03-2018_1m.csv  BTCUSDT_01-10-2017_1m.csv  BTCUSDT_02-03-2020_1m.csv
BTCUSDT_01-03-2019_1m.csv  BTCUSDT_01-10-2018_1m.csv  BTCUSDT_02-04-2018_1m.csv
BTCUSDT_01-03-2020_1m.csv  BTCUSDT_01-10-2019_1m.csv  BTCUSDT_02-04-2019_1m.csv
BTCUSDT_01-04-2018_1m.csv  BTCUSDT_01-10-2020_1m.csv  BTCUSDT_02-04-2020_1m.csv
BTCUSDT_01-04-2019_1m.csv  BTCUSDT_01-11

# --> Setup TPU

In [6]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print(tpu_strategy)
except ValueError:
  tpu_strategy = None
  #raise BaseException('ERROR: Not connected to a TPU runtime !')

Tensorflow version 2.4.1
Running on TPU  ['10.98.188.66:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.98.188.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.98.188.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# --> Classifying targets

In [ ]:
def classify(current, future):
    increase_percentage = ((float(future)-float(current))/float(current))
    if np.absolute(increase_percentage) < HOLD_PERCENT_DIFF:
        return 2 #2 is for holds
    elif float(future) > float(current):
        return 1 #1 is for buys
    else:
        return 0 #0 is for sells

# --> Data preprocessing

In [ ]:
def preprocess_df(df):
    for col in df.columns:                                
        if col != "target":                               #Don't preprocess targets
            df[col] = df[col].pct_change()                #Normalize data
            df.dropna(inplace=True)                       #Drops useless data
            df[col] = preprocessing.scale(df[col].values) #Scales data
    df.dropna(inplace=True)                               #Drops useless data                             
    
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)                     #Keeps adding data until max len
#     print(df.head(10))
#     print("===================================================================")
#     for c in df.columns:
#         print(c)
    for i in df.values:                                   #We take values, but not time since it's the index
        prev_days.append([n for n in i[:-1]])             #"n for n" is our rows without targets
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
    
    random.shuffle(sequential_data)                       #Shuffle data
    
    buys = []
    sells = []
    holds = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
        elif target == 2:
            holds.append([seq, target])
    random.shuffle(buys)
    random.shuffle(sells)
    random.shuffle(holds)
    lower = min(len(buys), len(sells), len(holds))
    buys = buys[:lower]                                   #Balance data
    sells = sells[:lower]
    holds = holds[:lower]
    sequential_data = buys + sells + holds
    random.shuffle(sequential_data)                       #Model shouldn't see only buys, then only sells...
    
    X = []
    y = []
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)

    return np.array(X), np.array(y)